# ESPIn Team Project - Flood Group
#### Ekta Aggarwal$^1$, Leia Barnes$^2$, Morgan Carrington$^3$, Prakash Pokhrel$^4$, Caroline Mierzejewski$^5$

[1] Univeristy, [2] University, [3] University, [4] University, [5] Texas State University

## Introduction
This project shows how overtime the intensity and frequency of rain events can cause incision and lateral changes in stream channels.

We...

This note